# Canadian fire


This notebook runs all the preprocessing steps using NRT data for a specific region and time. 

In [ ]:
# If you haven't installed the fireatlas code yet, uncomment the following line and run this cell.

!pip install -e .. -q

# After this runs, restart the notebook kernel.

In [ ]:
import datetime
import pandas as pd
import geopandas as gpd

from fireatlas import FireIO, FireMain, FireTime, FireObj, postprocess, preprocess, settings
from fireatlas.utils import timed

tst = [2023, 6, 1, 'AM']
ted = [2023, 8, 12, 'PM']
bbox = [-76.510259,  52.624922, -73.801554,  53.743852]
region = ('Issue2', bbox)
list_of_ts = list(FireTime.t_generator(tst, ted))

settings.OUTPUT_DIR = "take2"

## Preprocess the region

Preprocess the region to get rid of static flare sources. Save that new "swiss cheese" shape off into a geojson file for later.

In [ ]:
preprocess.preprocess_region(region)

## Are preprocessed input files available?

Check to see if all the files that we need have already been processed and are available on s3.

In [ ]:
ts_that_need_preprocessing = set([
    *preprocess.check_preprocessed_file(tst, ted, sat="NOAA20", freq="NRT", location="s3"),
    *preprocess.check_preprocessed_file(tst, ted, sat="SNPP", freq="NRT", location="s3")
])

In [ ]:
assert len(ts_that_need_preprocessing) == 0, "All input fires should be on s3"

## Preprocess for region and t

Do initial filtering and clustering using the preprocessed region (from local) and the half day files (from s3).

In [ ]:
%%time
for t in list_of_ts:
    preprocess.preprocess_region_t(t, region=region, read_location="s3", read_region_location="local")

## Run the fire expansion and merging algorithm

Now that we have all the region-specific preprocessed fires, we can go ahead and run `Fire_Forward`.

In [ ]:
%%time
allfires, allpixels = FireMain.Fire_Forward(tst=tst, ted=ted, restart=True, region=region, read_location="local")

## Take a look

In [ ]:
import folium
import hvplot.pandas

In [ ]:
region = preprocess.read_region(region, location="local")

gdf = postprocess.read_allfires_gdf(tst, ted, region, location="local").reset_index()
gdf = postprocess.fill_activefire_rows(gdf, ted)

gdf_fid = gdf[(gdf.fireID == 1) | (gdf.mergeid == 1)]
merged = postprocess.merge_rows(gdf_fid)

In [ ]:
mymap = folium.Map(max_bounds=region[1])
mymap.fit_bounds([[bbox[1], bbox[0]], [bbox[3], bbox[2]]])

# put the region on the map
folium.GeoJson(
    region[1],
    style_function=lambda x: {'fillColor': 'pink', 'color': 'red', 'weight': 2}
).add_to(mymap)

for i, t in enumerate(gdf.t.unique()[::-1]):
    data = merged[(merged["t"] == t)]
    folium.GeoJson(
        data["hull"],
        name=f"merged {t}",
        color="green" if i % 2 else "purple" ,
        fill_opacity=0.1,
        weight=2,
        show=False
    ).add_to(mymap)
    
    # add allfires perimeters
    data = gdf[(gdf["t"] == t) & (~gdf["invalid"])]
    folium.GeoJson(
        data["hull"],
        name=f"cluster hulls {t}",
        color="red" if i % 2 else "blue" ,
        fill_opacity=0.1,
        weight=2,
        show=False
    ).add_to(mymap)

mymap.add_child(folium.LayerControl(collapsed=False))

In [ ]:
merged[["t", "farea", "n_newpixels"]].hvplot(x="t", y="farea", secondary_y="n_newpixels")

## Write Snapshot files

Start by reading in the outputs from the run:

In [ ]:
allpixels = postprocess.read_allpixels(tst, ted, region, location="local")
allfires_gdf = postprocess.read_allfires_gdf(tst, ted, region, location="local")

Then iterate through all of the ts and save the snapshot layers.

In [ ]:
%%time
postprocess.save_snapshots(allfires_gdf, region, tst, ted)

## Write Largefire files

First we will find all the large fires:

In [ ]:
large_fires = postprocess.find_largefires(allfires_gdf)

In [ ]:
!zip -r data.zip ~/fireatlas_nrt/data/take2/Issue2/2023/Largefire/1

First we'll use the `allpixels` object to create the `nplist` layer

In [ ]:
%%time
postprocess.save_large_fires_nplist(allpixels, region, large_fires, tst)

The rest of the layers will be created directly from the `allfires_gdf` and have some merge handling

In [ ]:
%%time
postprocess.save_large_fires_layers(allfires_gdf, region, large_fires, tst, ted)